In [1]:
import pandas as pd
import random

import geopandas as gpd 
from shapely.geometry import Point, Polygon
import h3

import plotly.express as px

import json

In [2]:
data = pd.DataFrame(columns=['Latitude', 'Longitude'])

In [3]:
min_latitude = 43.5
max_latitude = 47.1
min_longitude = 6.6
max_longitude = 11.5

In [4]:
num_points = 1000000  # Cantidad de puntos a generar

lat = []
long = []

for _ in range(num_points):
    latitude = random.uniform(min_latitude, max_latitude)
    longitude = random.uniform(min_longitude, max_longitude)
    
    lat.append(latitude)
    long.append(longitude)

In [5]:
df = pd.DataFrame(list(zip(lat, long)),
               columns =['latitude', 'longitude'])

In [6]:
italia = gpd.read_file("data/ProvCM01012023_g_WGS84.shp")
italia = italia.loc[:,['geometry','COD_UTS','DEN_UTS']]
italia.geometry = italia.geometry.to_crs(epsg = 4326)
italia.head()

,geometry,COD_UTS,DEN_UTS
0,"POLYGON ((7.85904 45.59758, 7.86228 45.59699, ...",201,Torino
1,"POLYGON ((8.20447 45.93567, 8.21365 45.92490, ...",2,Vercelli
2,"POLYGON ((8.49688 45.83934, 8.49996 45.83402, ...",3,Novara
3,"MULTIPOLYGON (((7.46667 44.77289, 7.46997 44.7...",4,Cuneo
4,"POLYGON ((8.04681 45.12815, 8.04572 45.12300, ...",5,Asti


In [7]:
# Create a geometry column from lng & lat
df['geometry'] = df.apply(lambda x: Point(float(x.longitude), float(x.latitude)), axis=1)

# Create a GeoDataFrame from art and verify the type
df_points = gpd.GeoDataFrame(df, crs = italia.crs, geometry = df.geometry)
print(type(df_points))

df_points.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,latitude,longitude,geometry
0,44.930258,8.203329,POINT (8.20333 44.93026)
1,46.295594,7.894834,POINT (7.89483 46.29559)
2,44.153144,9.237340,POINT (9.23734 44.15314)
3,46.505900,9.991903,POINT (9.99190 46.50590)
4,45.779005,7.744820,POINT (7.74482 45.77901)


In [8]:
df_italia = gpd.sjoin(df_points, italia, op = 'within')
df_italia.drop(['index_right'], axis=1, inplace=True)
print(df_italia.shape)

/home/oecorrechag/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


(613095, 5)


In [9]:
# df_italia = df_italia[df_italia['DEN_RIP'] == 'Nord-Ovest']
# df_italia.head()

In [10]:
df_italia['latitude'].min(), df_italia['latitude'].max()

(43.50000925673767, 47.00979075076456)

In [11]:
df_italia['longitude'].min(), df_italia['longitude'].max()

(6.6280175101423, 11.499993426914166)

In [14]:
df_punticos = df_italia.copy()
df_punticos = df_punticos.loc[:,['latitude','longitude','DEN_UTS','COD_UTS']]
df_punticos.head()

,latitude,longitude,DEN_UTS,COD_UTS
0,44.930258,8.203329,Asti,5
91,44.945503,8.269372,Asti,5
196,44.972304,8.096594,Asti,5
204,44.831389,8.240147,Asti,5
447,44.838099,8.377956,Asti,5


In [15]:
df_punticos.to_csv('data/df_punticos.csv', encoding = 'utf-8-sig', index = False)

# Agregar las ciudades

In [16]:
# it = pd.read_csv('data/it.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
# it = it.loc[:,['lat','lng','city','population']]

# # Create a geometry column from lng & lat
# it['geometry'] = it.apply(lambda x: Point(float(x.lng), float(x.lat)), axis=1)

# # Create a GeoDataFrame from art and verify the type
# df_it = gpd.GeoDataFrame(it, crs = italia.crs, geometry = it.geometry)
# df_it = df_it.loc[:,['geometry','city','population']]
# print(type(df_it))

# df_it.head(20)

In [17]:
# df_italia.head()

In [18]:
# print(df_italia.shape)
# df_italia = gpd.sjoin(df_it, df_italia, op = 'within')
# print(df_italia.shape)

In [19]:
# # save
# df.to_csv('data/df.csv', encoding = 'utf-8-sig', index = False)

# Generar los H3

In [20]:
def get_h3_shape(row):
    h = h3.geo_to_h3(row["latitude"], row["longitude"], 7)
    return h

def get_geometry(h) :
    vertices = h3.h3_to_geo_boundary(h,  geo_json=True)
    polygon = Polygon(vertices)
    return polygon

# def get_geometry(h) :
#     h = h3.h3_to_geo_boundary(h, geo_json=True)
#     return Polygon(h)

In [21]:
df_italia["h3"] = df_italia.apply(get_h3_shape, axis=1)
df_italia["geometry"] = df_italia["h3"].apply(get_geometry)
df_italia.head()

,latitude,longitude,geometry,COD_UTS,DEN_UTS,h3
0,44.930258,8.203329,"POLYGON ((8.18576 44.94187, 8.18733 44.92945, ...",5,Asti,871f986adffffff
91,44.945503,8.269372,"POLYGON ((8.24772 44.94843, 8.24927 44.93601, ...",5,Asti,871f9861bffffff
196,44.972304,8.096594,"POLYGON ((8.08816 44.96921, 8.08974 44.95679, ...",5,Asti,871f9878bffffff
204,44.831389,8.240147,"POLYGON ((8.23076 44.83329, 8.23232 44.82085, ...",5,Asti,871f9bd25ffffff
447,44.838099,8.377956,"POLYGON ((8.35458 44.84632, 8.35612 44.83387, ...",5,Asti,871f986d8ffffff


In [22]:
type(df_italia)

geopandas.geodataframe.GeoDataFrame

# Agrupar los poligonos

In [23]:
df_poly = df_italia.copy()
df_poly = df_poly.loc[:,['h3','geometry','COD_UTS','DEN_UTS']]
df_poly.insert(0, 'count', 1)
print(df_poly.shape)
df_poly = df_poly.groupby(['h3','geometry','COD_UTS','DEN_UTS'], as_index=False).sum()
print(df_poly.shape)
df_poly.head()

(613095, 5)
(20889, 5)


,h3,geometry,COD_UTS,DEN_UTS,count
0,871ea0412ffffff,"POLYGON ((11.49877 44.46552, 11.49983 44.45289...",237,Bologna,2
1,871ea0414ffffff,"POLYGON ((11.49556 44.50340, 11.49663 44.49078...",237,Bologna,1
2,871ea0416ffffff,"POLYGON ((11.48143 44.48327, 11.48251 44.47064...",237,Bologna,16
3,871ea0430ffffff,"POLYGON ((11.49234 44.54127, 11.49341 44.52865...",237,Bologna,6
4,871ea0432ffffff,"POLYGON ((11.47821 44.52114, 11.47929 44.50852...",237,Bologna,24


In [24]:
df_poly = gpd.GeoDataFrame(df_poly, crs = italia.crs, geometry = df_poly.geometry)
type(df_poly)

geopandas.geodataframe.GeoDataFrame

In [25]:
df_poly.shape

(20889, 5)

In [26]:
df_poly['DEN_UTS'].unique()

array(['Bologna', 'Ferrara', 'Rovigo', 'Modena', 'Mantova', 'Verona',
       'Padova', 'Firenze', 'Arezzo', 'Prato', 'Pistoia', 'Siena', 'Pisa',
       'Lucca', 'Vicenza', 'Trento', 'Brescia', 'Bolzano',
       "Reggio nell'Emilia", 'Parma', 'Massa Carrara', 'La Spezia',
       'Cremona', 'Piacenza', 'Imperia', 'Savona', 'Cuneo', 'Livorno',
       'Genova', 'Sondrio', 'Bergamo', 'Vercelli', 'Biella', 'Torino',
       'Novara', 'Aosta', 'Pavia', 'Milano', 'Alessandria', 'Asti',
       'Verbano-Cusio-Ossola', 'Varese', 'Lodi', 'Lecco',
       'Monza e della Brianza', 'Como'], dtype=object)

In [27]:
df_poly = df_poly[df_poly['DEN_UTS'] == 'Milano']
df_poly = df_poly.loc[:,['COD_UTS','DEN_UTS','h3','geometry','count']]

In [28]:
df_poly.head()

,COD_UTS,DEN_UTS,h3,geometry,count
10096,215,Milano,871f98241ffffff,"POLYGON ((8.86056 45.35089, 8.86204 45.33849, ...",22
10101,215,Milano,871f98245ffffff,"POLYGON ((8.84271 45.36792, 8.84419 45.35552, ...",17
10103,215,Milano,871f98248ffffff,"POLYGON ((8.90969 45.33698, 8.91115 45.32458, ...",37
10104,215,Milano,871f98249ffffff,"POLYGON ((8.94097 45.34010, 8.94244 45.32769, ...",42
10106,215,Milano,871f9824affffff,"POLYGON ((8.89625 45.31681, 8.89772 45.30440, ...",2


In [29]:
df_poly.shape

(367, 5)

In [30]:
# save
df_poly.to_file('data/df_italia.geojson', driver='GeoJSON')  

In [ ]:
df_italia = gpd.GeoDataFrame(df_italia, crs = italia.crs, geometry = df_italia.geometry)
df_italia = df_italia.loc[:,['geometry', 'h3', 'COD_RIP']]
print(type(df_italia))
df_italia.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,geometry,h3,COD_RIP
1,"POLYGON ((8.70271 45.72135, 8.70293 45.71959, ...",891f98a4027ffff,1
24,"POLYGON ((9.33777 45.41386, 9.33797 45.41208, ...",891f9956aafffff,1
36,"POLYGON ((9.10932 44.97942, 9.10952 44.97764, ...",891f9bb32d3ffff,1
41,"POLYGON ((7.02920 45.89853, 7.02945 45.89679, ...",891f9c45d13ffff,1
46,"POLYGON ((8.96436 44.73538, 8.96456 44.73360, ...",891f9b17517ffff,1


In [ ]:
with open('data/df_italia2.geojson', 'r') as infile:
    geo_json = json.load(infile)

In [ ]:
geo_json

In [ ]:
fig = px.choropleth(df_italia2, geojson=geo_json, locations='COD_RIP', color='COD_RIP',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                        #    scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df_italia['DEN_RIP'].unique()

In [ ]:
type(df_italia)

In [ ]:
import pyproj

In [ ]:
df_italia.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)

In [ ]:
df_italia.head()

In [ ]:

fig = (
    px.scatter_mapbox(
        df_italia,
        lat="latitude",
        lon="longitude",
        color="COD_RIP",
        hover_data=["COD_RIP", "DEN_RIP"],
    )
    .update_traces(marker={"size": 10})
    .update_layout(
        mapbox={
            "style": "open-street-map",
            "zoom": 5,
            "layers": [
                {
                    "source": json.loads(df_italia.geometry2.to_json()),
                    "below": "traces",
                    "type": "line",
                    "color": "purple",
                    "line": {"width": 1.5},
                }
            ],
        },
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )
)
fig.show()

In [ ]:
print('Ok_')